In [149]:
%reload_ext autoreload
%autoreload 2
import sys
import pandas as pd
import tensorflow as tf

In [150]:
tf.get_logger().setLevel('ERROR') # only show error messages
from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
#from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_chrono_split,python_stratified_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items)
print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.8.18 (default, Sep 11 2023, 13:39:12) [MSC v.1916 64 bit (AMD64)]
Pandas version: 1.5.3
Tensorflow version: 2.13.0


In [151]:
users_df = pd.read_csv('BX-Users.csv', sep = ';', encoding = 'latin1', on_bad_lines = 'skip')
books_df = pd.read_csv('BX_Books.csv', sep = ';', encoding = 'latin1', on_bad_lines = 'skip')
ratings_df = pd.read_csv('BX-Book-Ratings.csv', sep = ';', encoding = 'latin1', on_bad_lines = 'skip')

In [152]:
ratings_df

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [153]:
books_df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271375,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271377,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [154]:
ISBN = books_df['ISBN'].tolist()

YearPublished = books_df['Year-Of-Publication'].tolist()


In [155]:
DateLookupDictionary = dict(zip(ISBN, YearPublished))

In [156]:
books_df['Book-Title'] = books_df['Book-Title'].dropna()
BookTitles = books_df['Book-Title'].tolist()
BookTitlesISBN = books_df['ISBN'].tolist()
BookLookupDict = dict(zip(BookTitlesISBN, BookTitles))

In [157]:
ratings_df['YearPublished'] = ratings_df['ISBN']
ratings_df

,User-ID,ISBN,Book-Rating,YearPublished
0,276725,034545104X,0,034545104X
1,276726,0155061224,5,0155061224
2,276727,0446520802,0,0446520802
3,276729,052165615X,3,052165615X
4,276729,0521795028,6,0521795028
...,...,...,...,...
1149775,276704,1563526298,9,1563526298
1149776,276706,0679447156,0,0679447156
1149777,276709,0515107662,10,0515107662
1149778,276721,0590442449,10,0590442449


In [158]:
def ISBNRemover(x):
    if x not in DateLookupDictionary:
        return None
    else:
        return DateLookupDictionary[x]
    
def BookChecker(x):
    if str(x) not in BookLookupDict:
        return None
    else:
        return x

In [159]:
ratings_df['YearPublished'] = ratings_df['YearPublished'].apply(ISBNRemover)
ratings_df["ISBN"] = ratings_df["ISBN"].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

In [162]:
ratings_df = ratings_df[ratings_df['YearPublished'].notna()]
ratings_df = ratings_df[ratings_df['ISBN'].notna()]

In [164]:
ratings_df['ISBN'] = ratings_df['ISBN'].apply(lambda x: int(float(x)))
ratings_df['YearPublished'] = ratings_df['YearPublished'].astype(int)
ratings_df['ISBN'] = ratings_df['ISBN'].apply(BookChecker)
ratings_df = ratings_df[ratings_df['ISBN'].notna()]
ratings_df

In [168]:
ratings_df = ratings_df.rename(columns={
    'User-ID':"userID",'ISBN':"itemID",'Book-Rating':"rating",'YearPublished':"timestamp"
})
ratings_df

,userID,itemID,rating,timestamp
5,276733,2.080675e+09,0,1998
22,276747,1.570231e+09,0,1999
23,276747,1.885408e+09,7,1998
27,276751,3.596218e+09,8,1998
37,276762,3.404123e+09,0,1995
...,...,...,...,...
1149651,276681,1.931333e+09,10,2003
1149746,276688,1.551669e+09,6,2002
1149747,276688,1.575661e+09,7,1996
1149748,276688,1.580601e+09,0,1996


In [169]:
df = ratings_df.head(10000)
df

,userID,itemID,rating,timestamp
5,276733,2.080675e+09,0,1998
22,276747,1.570231e+09,0,1999
23,276747,1.885408e+09,7,1998
27,276751,3.596218e+09,8,1998
37,276762,3.404123e+09,0,1995
...,...,...,...,...
87208,18850,3.442723e+09,0,1998
87209,18850,3.442723e+09,5,1998
87210,18850,3.442727e+09,0,2001
87211,18850,3.453023e+09,0,1986


In [170]:
users = df['userID'].unique().tolist()
items = df['itemID'].unique().tolist()

ref = [i for i in range(1, 50001)]
userDict = dict(zip(users, ref))
itemDict = dict(zip(items, ref))

userDictRev = dict(zip(ref, users))
itemDictRev = dict(zip(ref, items))

In [171]:
df['userID'] = df['userID'].apply(lambda x: userDict[x])
df['itemID'] = df['itemID'].apply(lambda x: itemDict[x])
df

C:\Users\yoges\AppData\Local\Temp\ipykernel_6296\976861918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['userID'] = df['userID'].apply(lambda x: userDict[x])
C:\Users\yoges\AppData\Local\Temp\ipykernel_6296\976861918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['itemID'] = df['itemID'].apply(lambda x: itemDict[x])


,userID,itemID,rating,timestamp
5,1,1,0,1998
22,2,2,0,1999
23,2,3,7,1998
27,3,4,8,1998
37,4,5,0,1995
...,...,...,...,...
87208,1930,7995,0,1998
87209,1930,7996,5,1998
87210,1930,7997,0,2001
87211,1930,4104,0,1986


In [172]:
train, test = python_chrono_split(df, 0.75)
train_file = "./train.csv"
test_file = "./test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

In [173]:
TOP_K = 10
# Model parameters
EPOCHS = 50
BATCH_SIZE = 256
SEED = 42

In [174]:
data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED)

INFO:recommenders.models.ncf.dataset:Indexing ./train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test.csv ...
INFO:recommenders.models.ncf.dataset:Creating full leave-one-out test file ./test_full.csv ...
100%|███████████████████████████████████████████████████████████████████████████████| 473/473 [00:03<00:00, 135.40it/s]
INFO:recommenders.models.ncf.dataset:Indexing ./test_full.csv ...


In [175]:
model = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=0.002,
    verbose=5,
    seed=SEED
)

C:\Users\yoges\anaconda3\envs\test\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [176]:
with Timer() as train_time:
    model.fit(data)
print("Took {} seconds for training.".format(train_time))
#adding timimg for predictions
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))
    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})
    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)
print("Took {} seconds for prediction.".format(test_time))

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 5 [5.05s]: train_loss = 0.209366 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [5.04s]: train_loss = 0.131475 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 15 [8.42s]: train_loss = 0.089939 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [4.91s]: train_loss = 0.068669 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 25 [5.40s]: train_loss = 0.050585 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 30 [4.79s]: train_loss = 0.036550 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 35 [5.02s]: train_loss = 0.029756 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 40 [5.01s]: train_loss = 0.024011 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 45 [5.90s]: train_loss = 0.019544 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 50 [5.55s]: train_loss = 0.016183 


Took 264.9191 seconds for training.
Took 114.3445 seconds for prediction.


In [177]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.000277
NDCG:	0.000871
Precision@K:	0.000423
Recall@K:	0.002643


In [181]:
all_predictions = all_predictions[['userID','itemID','prediction']]
# rename columns
all_predictions = all_predictions.rename(columns={
   "userID":'CustomerID',"itemID":'ISBN',"rating":'Quantity','prediction':'probability'
})

all_predictions['CustomerID'] = all_predictions['CustomerID'].apply(lambda x: userDictRev[x])
all_predictions['ISBN'] = all_predictions['ISBN'].apply(lambda x: itemDictRev[x])
df2 = ratings_df.head(10000)

all_predictions

,CustomerID,StockCode,probability
7903,1,3,1.926282e-07
7904,1,2,1.276547e-10
7905,1,4,2.744335e-07
7906,1,8,5.577149e-11
7907,1,15,1.981446e-10
...,...,...,...
12523765,1930,7976,6.461463e-08
12523766,1930,7977,7.732265e-09
12523767,1930,7978,1.936315e-06
12523768,1930,7979,1.866030e-08


In [243]:
def recommendation_system(customer_id):
    print(" \n---------- Top 5 Bought Items -----------\n")
    print(df2[df2['userID'] == customer_id][['userID', 'itemID','rating']].nlargest(5, 'rating'))
    top_5 = (df2[df2['userID'] == customer_id][['userID', 'itemID','rating']].nlargest(5, 'rating'))
    print('\n-------Top 5 Book Names------\n')
    for i in range(5):
        print(books_df[books_df['ISBN'] == str(int(top_5['itemID'].iloc[i]))]['Book-Title'].iloc[0])
    print("\n --------- Top 5 Recommendations ------------ \n")
    print(all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability'))
    recommend = all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability')
    print('\n-------Top 5 Book Name of Recommendations ------\n')
    for i in range(5):
        print(books_df[books_df['ISBN'] == str(int(recommend['ISBN'].iloc[i]))]['Book-Title'].iloc[0])

In [244]:
recommendation_system(18850)

 
---------- Top 5 Bought Items -----------

       userID        itemID  rating
87193   18850  3.423014e+09      10
87197   18850  3.426617e+09      10
87186   18850  3.257219e+09       9
87207   18850  3.442542e+09       8
87185   18850  3.257215e+09       7

-------Top 5 Book Names------

Das Versprechen. Requiem auf den Kriminalroman.
About a Boy
Tod Im Herbst
Russendisko.
Selbs Justiz. Roman.

 --------- Top 5 Recommendations ------------ 

          CustomerID          ISBN  probability
12522350       18850  8.495501e+09     0.935218
12518294       18850  2.020292e+09     0.858490
12521957       18850  1.858287e+09     0.856117
12520062       18850  2.228128e+09     0.816385
12519543       18850  1.558749e+09     0.787955

-------Top 5 Book Name of Recommendations ------

Mujer En Guerra - Bolsillo
Je suis vivant et vous Ã?Âªtes morts
The Mini Rough Guide to Paris
Dire et interdire: Ã©lÃ©ments de jurologie (Collection Langages et sociÃ©tÃ©s)
Chicken Soup for the Gardener's Soul, 